In [1]:
import pandas as pd 
import numpy as np

In [28]:
df = pd.read_csv('블로그감성분석.csv')

In [56]:
ndf = df.pivot_table(index = '검색키워드', values='감성분석결과',aggfunc=['count','sum'])

In [57]:
ndf.columns = ['전체','긍정']
ndf['부정'] = ndf.전체 - ndf.긍정
ndf['긍정비율'] = round(ndf.긍정 / ndf.전체,2)  
ndf['부정비율'] = round(ndf.부정 / ndf.전체,2)

In [62]:
ndf.sort_values('긍정비율', ascending=False,inplace=True)
ndf

,전체,긍정,부정,긍정비율,부정비율
검색키워드,,,,,
토이빌리지 대구혁신점,28,28,0,1.00,0.00
운암지먹거리타운,30,30,0,1.00,0.00
마비정벽화마을,26,26,0,1.00,0.00
동인동 찜갈비골목,29,28,1,0.97,0.03
화본역 급수탑,29,28,1,0.97,0.03
...,...,...,...,...,...
월곡역사공원,30,6,24,0.20,0.80
장곡자연휴양림,20,4,16,0.20,0.80
나비생태원,27,5,22,0.19,0.81


In [95]:
ndf.to_csv('긍정부정.csv')

In [69]:
len(ndf[ndf['긍정비율'] > 0.9])

17

### Word2Vec

In [90]:
from tqdm import tqdm

In [74]:
texts= df['data'].tolist()

In [91]:
for i in tqdm(range(len(texts))):
    texts[i] = texts[i].split()

100%|███████████████████████████████████████████████████████████████████████████| 7560/7560 [00:00<00:00, 15636.76it/s]


In [96]:
len(texts)

7560

In [97]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
model = Word2Vec(sentences=texts, vector_size=100, window=10, min_count=5, sg=0)

In [98]:
model.save("my_word2vec_model.model")

In [115]:
loaded_model = Word2Vec.load("my_word2vec_model.model")

In [116]:
# 단어 벡터 검색
vector = loaded_model.wv['장미']

In [120]:
# 유사한 단어 찾기
similar_words = loaded_model.wv.most_similar('카페')
print(similar_words)

[('커피', 0.7492390275001526), ('카페인', 0.7389590740203857), ('카페나', 0.658656656742096), ('에이비오', 0.6426032185554504), ('커피한잔', 0.6371686458587646), ('베이커리', 0.6358869075775146), ('테이크아웃', 0.6309372782707214), ('디저트', 0.6234607696533203), ('신상', 0.6174972057342529), ('브런치', 0.6168152689933777)]


### doc2vec

In [121]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [123]:
texts= df['data'].tolist()

In [151]:
# 데이터 준비
documents = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(texts)]

# 모델 초기화
model = Doc2Vec(vector_size=100, window=20, min_count=1, workers=4)

# 모델 학습
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=50)

0it [00:00, ?it/s]

In [154]:
model.save("doc2vec.model")

In [166]:
similar_documents = model.dv.most_similar(positive=[model.infer_vector(['대구에서 막창먹고 싶어'])], topn = 5)
similar_documents

[('3132', 0.3135724663734436),
 ('2950', 0.2973553240299225),
 ('5053', 0.2829011380672455),
 ('7480', 0.2763986885547638),
 ('6574', 0.26972419023513794)]

In [167]:
idx=[int(i[0]) for i in similar_documents]
idx

[3132, 2950, 5053, 7480, 6574]

In [168]:
df.iloc[idx[0]]['링크'], df.iloc[idx[1]]['링크'], df.iloc[idx[2]]['링크'], df.iloc[idx[3]]['링크'], df.iloc[idx[4]]['링크']

('https://blog.naver.com/godkjh2319/223028467626',
 'https://blog.naver.com/daegu_news/222518361493',
 'https://blog.naver.com/dalseoblog/223097497765',
 'https://blog.naver.com/rozaniz641/223127818047',
 'https://blog.naver.com/jho8027/223221901635')